# Weaviate Transformation Agent - Workshop

### Prerequisites

1. Log in to [Weaviate Cloud](https://console.weaviate.cloud) account (sign up if you don't have one yet)
1. Create a Weaviate Cloud [Sandbox](https://weaviate.io/developers/wcs/manage-clusters/create#sandbox-clusters) instance
1. Go to the 'Embedding' tab (on the left column) and enable `Weaviate Embeddings`
1. Take note of the `REST Endpoint` and a `Admin` `API Key`. 
1. Update `WEAVIATE_CLOUD_URL` with the `REST Endpoint` and `WEAVIATE_API_KEY` with the `Admin` `API Key` in the `.env` file in the root directory of this repository.

## Introduction

### Agenda

Let's talk about:
- What the Transformation Agent is
- What you can do with the Transformation Agent
- Some tips & tricks
- How to get started

### About the Transformation Agent

The *Weaviate Transformation Agent* is 

- A cloud-based service 
- for transforming your data in a Weaviate instance
- available for Weaviate Cloud users

**And** it is: in technical preview (do **not** use in production)

<center><img src="img/agents_tech_preview.png" width="60%"></center>

> ⚠️ The Weaviate Transformation Agent modifies data objects in Weaviate. **While the Agent is in technical preview, do not use it in a production environment.** 
> 
> The Agent may not work as expected, and the data in your Weaviate instance may be affected in unexpected ways.

**What the Transformation Agent is**

<center><img src="img/ta_obj.png" width="60%"></center>

The `TransformationAgent` can modify objects in a Weaviate collection to add new properties or update existing properties.

**What you can do with the Transformation Agent**

<center><img src="img/ta_overview.png" width="60%"></center>

Provide instructions to the `TransformationAgent` using natural language, and other required parameters. 

## Preparation

Here, we are going to use the [**Weaviate/ArxivPapers**](https://huggingface.co/datasets/weaviate/agents/viewer/query-agent-ecommerce) dataset. 

It includes titles and abstracts of a few research papers.

First, we load the dataset & add it to Weaviate.

### Load dataset

In [1]:
from datasets import load_dataset

papers_dataset = load_dataset("weaviate/agents", "transformation-agent-papers", split="train")

In [2]:
print(papers_dataset.shape)
print(papers_dataset[0]["properties"].keys())

(2000, 2)
dict_keys(['abstract', 'title'])


In [3]:
for k, v in papers_dataset[0]["properties"].items():
    if len(v) > 100:
        v = v[:100] + "..."
    print(f"{k}: {v}")

abstract:   Astronomy is increasingly encountering two fundamental truths: (1) The field
is faced with the tas...
title: Discussion on "Techniques for Massive-Data Machine Learning in
  Astronomy" by A. Gray


Iterate through the data

In [4]:
columns = papers_dataset[0]["properties"].keys()

for i, item in enumerate(papers_dataset):
    if i < 2:
        properties = {
            col: item["properties"][col] for col in columns
        }
        print(properties)

{'abstract': "  Astronomy is increasingly encountering two fundamental truths: (1) The field\nis faced with the task of extracting useful information from extremely large,\ncomplex, and high dimensional datasets; (2) The techniques of astroinformatics\nand astrostatistics are the only way to make this tractable, and bring the\nrequired level of sophistication to the analysis. Thus, an approach which\nprovides these tools in a way that scales to these datasets is not just\ndesirable, it is vital. The expertise required spans not just astronomy, but\nalso computer science, statistics, and informatics. As a computer scientist and\nexpert in machine learning, Alex's contribution of expertise and a large number\nof fast algorithms designed to scale to large datasets, is extremely welcome.\nWe focus in this discussion on the questions raised by the practical\napplication of these algorithms to real astronomical datasets. That is, what is\nneeded to maximally leverage their potential to impro

### Ingest data into Weaviate

#### Connect to Weaviate

In [5]:
import os
import dotenv

dotenv.load_dotenv()

# Update the variables in the .env file with your own values
weaviate_url = os.getenv("WEAVIATE_CLOUD_URL")
weaviate_api_key = os.getenv("WEAVIATE_CLOUD_API_KEY")

In [6]:
weaviate_url

'https://1ree7zierqqrwwnif6b6ug.c0.europe-west3.gcp.weaviate.cloud'

In [7]:
import weaviate
from weaviate.classes.init import Auth

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=weaviate_url, auth_credentials=Auth.api_key(weaviate_api_key)
)

assert client.is_ready()

/Users/jphwang/code/demo-code/weaviate-agents/transformation/.venv/lib/python3.12/site-packages/weaviate/warnings.py:340: UserWarning: Con006: You're using the sync client in an async context. This usage is discouraged to avoid blocking your async event loop with sync I/O calls.
            We encourage you to update your code to use the async client instead when running inside async def functions!
  warnings.warn(


#### Set up a collection

**Important:** Make sure to enable 'Embeddings' in the Weaviate Cloud console. 

[See above](#prerequisites)

In [8]:
from weaviate.classes.config import Configure, Property, DataType

collection_name = "ArxivPapersDemo"

# Can delete the collection if you would like to (re)start fresh
client.collections.delete(collection_name)

if client.collections.exists(collection_name):
    # For re-running this tutorial, do nothing
    pass
else:
    client.collections.create(
        collection_name,
        description="A dataset that lists research paper titles and abstracts",
        properties=[
            Property(name="title", data_type=DataType.TEXT),
            Property(name="abstract", data_type=DataType.TEXT),
        ],
        vectorizer_config=[
            Configure.NamedVectors.text2vec_weaviate(
                name="default",
                source_properties=["title", "abstract"],
            )
        ]
    )

#### Add data to Weaviate

We loop through the data and add it to Weaviate. 

For the demo/workshop, we add only a few rows for speed and simplicity.

In [9]:
papers_collection = client.collections.get(collection_name)
columns = papers_dataset[0]["properties"].keys()

with papers_collection.batch.fixed_size(100) as batch:
    for i, item in enumerate(papers_dataset):
        if i < 50:
            properties = {col: item["properties"][col] for col in columns}
            batch.add_object(properties=properties)


if papers_collection.batch.failed_objects:
    for fo in papers_collection.batch.failed_objects[:3]:
        print(fo.message)
        print(fo.object_)

In [10]:
len(papers_collection)

50

#### Inspect the collection 



In [11]:
response = papers_collection.query.fetch_objects(
    limit=3,
    include_vector=True
)

for o in response.objects:
    for k, v in o.properties.items():
        print(f"{k}: {v[:50]}")
    print()
    print(o.vector)

abstract:   Data from spectrophotometers form vectors of a l
title: Mutual information for the selection of relevant v

{'default': [-0.049957275390625, -0.006366729736328125, -0.042755126953125, 0.05865478515625, -0.063720703125, -0.041595458984375, -0.007244110107421875, -0.0239715576171875, -0.0149383544921875, -0.064208984375, -0.031524658203125, -0.07513427734375, 0.0614013671875, -0.0030574798583984375, -0.0185699462890625, 0.07373046875, -0.046661376953125, 0.0535888671875, 0.01776123046875, -0.00624847412109375, 0.0017900466918945312, 0.004535675048828125, -0.07879638671875, -0.021636962890625, 0.0090789794921875, -0.071533203125, -0.024261474609375, -0.00894927978515625, 0.00972747802734375, 0.07867431640625, -0.023193359375, 0.00014090538024902344, -0.004047393798828125, -0.030975341796875, 0.0016384124755859375, 0.080810546875, -0.03173828125, -0.0275421142578125, -0.0129852294921875, 0.0304412841796875, -0.0176849365234375, -0.016204833984375, 0.0028553009033203125, -0.1334

**Alternative: Use the `Explorer` cloud tool**

On Weaviate Cloud Console, click on the `Explorer` tab on the left column.

When you click on each object, you should see 2 properties:
- `title`
- `abstract`

As well as its `vectors`

## Using the original dataset:


### Can you find what you need?

Can you find papers about a specific topic (e.g. machine learning)?

In [12]:
response = papers_collection.query.near_text(
    query="machine learning",
    limit=5
)

for o in response.objects:
    print(o.properties["title"])

Probabilistic Approach to Neural Networks Computation Based on Quantum
  Probability Model Probabilistic Principal Subspace Analysis Example
Efficient Bayes-Adaptive Reinforcement Learning using Sample-Based
  Search
Discussion on "Techniques for Massive-Data Machine Learning in
  Astronomy" by A. Gray
Transfer Learning Using Feature Selection
Bayesian Active Learning for Classification and Preference Learning


Can you filter only for papers that are reviews/surveys?

In [13]:
## ???

### Does your data meet your needs?

What if: 
- The data is in the wrong language?
- Each abstract is too long?

Would you want to perform a RAG query each time?




## Try the Weaviate Transformation Agent 

### Task 1: Create a `topics` property

Define the operation(s) that you want to perform on the data.

In [14]:
prompt_create_topics = """
Create a list of topic tags based on the abstract.
Topics should be distinct from each other. Provide a maximum of 5 topics.
Group similar topics under one topic tag.
"""

In [15]:
from weaviate.agents.classes import Operations

add_topics = Operations.append_property(
    property_name="topics",             # Property to create
    data_type=DataType.TEXT_ARRAY,      # Data type of the property
    view_properties=["abstract"],       # Existing properties to view for the operation
    instruction=prompt_create_topics,   # Instruction to the Transformation Agent
)

Instantiate the agent & start the operations

In [16]:
from weaviate.agents.transformation import TransformationAgent

ta = TransformationAgent(
    client=client,              # Weaviate client object
    collection=collection_name, # Collection name
    operations=[add_topics]     # List of transform operations
)

ta_response = ta.update_all()

What does the response look like?

In [17]:
ta_response

TransformationResponse(workflow_id='TransformationWorkflow-146fb8e963f0b6da6c07997f09f0ecad')

The response contains the unique `workflow_id` of the operations. 

This does not mean that the operations are finished!

**The Transformation Agent is asynchronous**. You can check the status of the operation using the `workflow_id`.

In [18]:
ta.get_status(workflow_id=ta_response.workflow_id)

{'workflow_id': 'TransformationWorkflow-146fb8e963f0b6da6c07997f09f0ecad',
 'status': {'batch_count': 1,
  'end_time': None,
  'start_time': '2025-03-25 12:09:56',
  'state': 'running',
  'total_duration': None,
  'total_items': 50}}

We can periodically check if the operation is done

In [19]:
def get_ta_status(agent_instance, workflow_id):
    # Rough code to check the status of the TA workflow
    import time
    from datetime import datetime

    while True:
        status = agent_instance.get_status(workflow_id=workflow_id)

        if status["status"]["state"] != "running":
            break

        # Calculate elapsed time from start_time
        start = datetime.strptime(status["status"]["start_time"], "%Y-%m-%d %H:%M:%S")
        elapsed = (datetime.now() - start).total_seconds()

        print(f"Waiting... Elapsed time: {elapsed:.2f} seconds")
        time.sleep(10)

    # Calculate total time
    if status["status"]["total_duration"]:
        total = status["status"]["total_duration"]
    else:
        start = datetime.strptime(status["status"]["start_time"], "%Y-%m-%d %H:%M:%S")
        end = datetime.now() if not status["status"]["end_time"] else datetime.strptime(status["status"]["end_time"], "%Y-%m-%d %H:%M:%S")
        total = (end - start).total_seconds()

    print(f"Total time: {total:.2f} seconds")
    print(status)

In [20]:
get_ta_status(agent_instance=ta, workflow_id=ta_response.workflow_id)

Waiting... Elapsed time: 36.36 seconds
Waiting... Elapsed time: 47.00 seconds
Total time: 53.63 seconds
{'workflow_id': 'TransformationWorkflow-146fb8e963f0b6da6c07997f09f0ecad', 'status': {'batch_count': 1, 'end_time': '2025-03-25 12:10:50', 'start_time': '2025-03-25 12:09:56', 'state': 'completed', 'total_duration': 53.626265, 'total_items': 50}}


**How the Transformation Agent works**

<center><img src="img/ta_schematic.png" width="60%"></center>

The `TransformationAgent` connects to your Weaviate Cloud instance, and uses LLMs to follow these instructions.

When the operation is complete - let's see what we can do with the data:

In [21]:
from weaviate.classes.query import Metrics

response = papers_collection.aggregate.over_all(
    return_metrics=Metrics("topics").text(
        top_occurrences_count=True,
        top_occurrences_value=True,
        min_occurrences=10
    )
)

for t in response.properties["topics"].top_occurrences:
    print(t)

TopOccurrence(count=31, value='Machine Learning')
TopOccurrence(count=7, value='Data Analysis')
TopOccurrence(count=5, value='Algorithms')
TopOccurrence(count=5, value='Artificial Intelligence')
TopOccurrence(count=5, value='Computer Science')
TopOccurrence(count=4, value='Graph Theory')
TopOccurrence(count=4, value='Optimization')
TopOccurrence(count=4, value='Reinforcement Learning')
TopOccurrence(count=4, value='Statistics')
TopOccurrence(count=3, value='Clustering')


Try to filter for papers with particular topics:

In [22]:
# Filter for papers

Inspect an object again:

In [23]:
response = papers_collection.query.fetch_objects(
    limit=3,
)

for o in response.objects:
    for k, v in o.properties.items():
        print(f"{k}: {v[:50]}")
    print()

abstract:   Data from spectrophotometers form vectors of a l
title: Mutual information for the selection of relevant v
topics: ['Machine Learning', 'Data Analysis', 'Model Optimization', 'Variable Selection', 'Nonlinear Modelling']

abstract:   We consider finite horizon Markov decision proce
title: Mean-Variance Optimization in Markov Decision Proc
topics: ['Markov Decision Processes', 'Optimization', 'Algorithm Design', 'Policies', 'Computational Complexity']

abstract:   Unlike static documents, version controlled docu
title: Local Space-Time Smoothing for Version Controlled 
topics: ['Collaboration', 'Document Revision', 'Modeling and Visualization', 'Data Management', 'Natural Language Processing']



### Task 2: Perform multiple operations

- Add a `paper_type` property (e.g. `survey`, `method`, `resource`)
- Add a boolean property `relevant_to_rag` (True/False)

In [24]:
prompt_paper_type = """
Determine the primary type of paper based on the abstract. Assign exactly one of the following categories that best represents the paper's main contribution:

'survey':   Comprehensive review or meta-analysis of existing work in a field
'model':    Introduction of a new predictive model, statistical method, or algorithmic approach
'system':   Description of a new data pipeline, workflow, framework, or system architecture
'analysis': Focused on insights derived from analyzing data
'resource': Introduction of a new dataset, benchmark, or tool for data science
'other':    None of the above
"""

add_paper_type = Operations.append_property(
      property_name="paper_type",
      data_type=DataType.TEXT,
      view_properties=["abstract"],
      instruction=prompt_paper_type,
)

In [25]:
prompt_about_classification = """
Based on the abstract, determine whether the paper is
primarily about the machine field of classification.

Do not include papers that are obliquely, or vaguely about classification.
"""

add_about_classification_bool = Operations.append_property(
    property_name="about_classification",
    data_type=DataType.BOOL,
    view_properties=["abstract"],
    instruction=prompt_about_classification,
)

In [26]:
prompt_add_french_title_suffix = """
Update the title to ensure that it contains the French translation of itself in parantheses, after the original title.
"""

update_title = Operations.update_property(
    property_name="title",
    view_properties=["title"],
    instruction=prompt_add_french_title_suffix,
)

In [27]:
from weaviate.agents.transformation import TransformationAgent

ta = TransformationAgent(
    client=client,
    collection=collection_name,
    operations=[
        update_title,
        add_paper_type,
        add_about_classification_bool
    ],
)

ta_response = ta.update_all()

In [28]:
ta.get_status(workflow_id=ta_response.workflow_id)

{'workflow_id': 'TransformationWorkflow-463cce1e2f29563e51607e60d2a747b9',
 'status': {'batch_count': 1,
  'end_time': None,
  'start_time': '2025-03-25 12:11:09',
  'state': 'running',
  'total_duration': None,
  'total_items': 50}}

In [29]:
get_ta_status(agent_instance=ta, workflow_id=ta_response.workflow_id)

Waiting... Elapsed time: 4.05 seconds
Waiting... Elapsed time: 14.69 seconds
Total time: 13.83 seconds
{'workflow_id': 'TransformationWorkflow-463cce1e2f29563e51607e60d2a747b9', 'status': {'batch_count': 1, 'end_time': '2025-03-25 12:11:23', 'start_time': '2025-03-25 12:11:09', 'state': 'completed', 'total_duration': 13.826357, 'total_items': 50}}


Inspect a few transformed objects:

In [30]:
response = papers_collection.query.fetch_objects(
    limit=3,
)

for o in response.objects:
    for k, v in o.properties.items():
        if type(v) == str:
            if len(v) > 50:
                v = v[:50] + "..."
        print(f"{k}: {v}")
    print()

abstract:   Data from spectrophotometers form vectors of a l...
title: Mutual information for the selection of relevant v...
topics: ['Machine Learning', 'Data Analysis', 'Model Optimization', 'Variable Selection', 'Nonlinear Modelling']
paper_type: analysis
about_classification: False

abstract:   We consider finite horizon Markov decision proce...
topics: ['Markov Decision Processes', 'Optimization', 'Algorithm Design', 'Policies', 'Computational Complexity']
paper_type: analysis
title: Mean-Variance Optimization in Markov Decision Proc...
about_classification: False

abstract:   Unlike static documents, version controlled docu...
topics: ['Collaboration', 'Document Revision', 'Modeling and Visualization', 'Data Management', 'Natural Language Processing']
paper_type: None
title: Local Space-Time Smoothing for Version Controlled ...
about_classification: False



Perform new queries - e.g. what paper types do we have?

In [31]:
from weaviate.classes.query import Metrics

response = papers_collection.aggregate.over_all(
    return_metrics=Metrics("paper_type").text(
        top_occurrences_count=True,
        top_occurrences_value=True,
        min_occurrences=10
    )
)

for t in response.properties["paper_type"].top_occurrences:
    print(t)

TopOccurrence(count=27, value='model')
TopOccurrence(count=18, value='analysis')
TopOccurrence(count=2, value='other')
TopOccurrence(count=1, value='survey')


How many objects are relevant to language models?

In [32]:
from weaviate.classes.query import Filter

response = papers_collection.aggregate.over_all(
    filters=Filter.by_property("about_classification").equal(True),
)

response.total_count

11

In [33]:
from weaviate.classes.query import Filter

response = papers_collection.query.fetch_objects(
    filters=Filter.by_property("about_classification").equal(True),
    limit=10
)

for o in response.objects:
    print(o.properties["title"])

Classification under Data Contamination with Application to Remote Sensing Image Mis-registration
Fast Inference in Sparse Coding Algorithms with Applications to Object Recognition (Inferer dans les Algorithmes de Codage Eparse avec Applications à la Reconnaissance d'Objets)
Transfer Learning Using Feature Selection (Apprentissage transfert à l'aide de la sélection des caractéristiques)
Distribution-Specific Agnostic Boosting
Using a Kernel Adatron for Object Classification with RCS Data (Utilisation d'une Adatron de noyau pour la classification d'objet avec des données RCS)
Low congestion online routing and an improved mistake bound for online prediction of graph labeling (Routing et prediction en ligne avec une congestion réduite et une limite d'erreur améliorée pour l'étiquetage de graphe)
A Spectral Algorithm for Latent Dirichlet Allocation (Un algorithme spectral pour l'allocation Dirichlet latente)
Using Genetic Algorithms for Texts Classification Problems (Utilisation des algori

In [34]:
from weaviate.classes.query import Filter

response = papers_collection.aggregate.over_all(
    filters=(
        Filter.by_property("paper_type").equal("model")
        &
        Filter.by_property("about_classification").equal(True)
    )
)

response.total_count

6

In [35]:
from weaviate.classes.query import Filter

response = papers_collection.query.near_text(
    query="vector",
    filters=(
        Filter.by_property("paper_type").equal("model")
        &
        Filter.by_property("about_classification").equal(True)
    )
)

for o in response.objects:
    print(o.properties["title"])

Using a Kernel Adatron for Object Classification with RCS Data (Utilisation d'une Adatron de noyau pour la classification d'objet avec des données RCS)
A Spectral Algorithm for Latent Dirichlet Allocation (Un algorithme spectral pour l'allocation Dirichlet latente)
Fast Inference in Sparse Coding Algorithms with Applications to Object Recognition (Inferer dans les Algorithmes de Codage Eparse avec Applications à la Reconnaissance d'Objets)
Distribution-Specific Agnostic Boosting
Bayesian Active Distance Metric Learning (Apprentissage de la distance métrique active Bayesian)
Bayesian Active Learning for Classification and Preference Learning (Apprentissage actif bayésien pour la classification et l'apprentissage des préférences)


## Bonus: Use the Query Agent

In [36]:
from weaviate.agents.query import QueryAgent

qa = QueryAgent(
    client=client, collections=[collection_name]
)

In [41]:
# Perform a query
response = qa.run(
    """
    Find papers that are about classification. Tell me about some of them.
    Hint: There is a property called 'about_classification' that you can use.
    """,
)

# Print the response
response.display()

╭─────────────────────────────────────────────── 🔍 Original Query ───────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│     Find papers that are about classification. Tell me about some of them.                                      │
│     Hint: There is a property called 'about_classification' that you can use.                                   │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📝 Final Answer ────────────────────────────────────────────────╮
│                                                                                                                 │
│ Here are some papers related to classification:                                                                 │
│                                                                                                                 │
│ 1. **Using Genetic Algorithms for Texts Classification Problems**                                               │
│    - This paper discusses the application of genetic algorithms to solve text classification problems, which is │
│ a subfield of data mining. Classification here involves determining the belonging of text to pre-defined        │
│ classes, and clustering involves dividing text sets into clusters based on their natural similarities.          │
│                                                                                                                 │
│ 2. **Using a Kernel Adatron for Object Classification with RCS Data**                                           │
│    - This study explores the classification of objects using Radar Cross Section (RCS) signals with support     │
│ vector machines (SVMs). The paper compares SVM efficiency to Bayesian approaches, highlighting SVM's superior   │
│ speed and performance in classifying different shapes using synthesized RCS data.                               │
│                                                                                                                 │
│ 3. **Bayesian Active Learning for Classification and Preference Learning**                                      │
│    - This paper presents an approach to active learning in the context of Gaussian Process Classifiers (GPCs),  │
│ focusing on minimizing information loss during classification tasks. It also expands on binary preference       │
│ learning via classification.                                                                                    │
│                                                                                                                 │
│ 4. **Optimizing F-measure: A Tale of Two Approaches**                                                           │
│    - This research analyzes two methods for improving classifiers’ F-measures: empirical utility maximization   │
│ and decision-theoretic approaches, offering insights into which method is preferable under various conditions.  │
│                                                                                                                 │
│ 5. **Classification under Data Contamination with Application to Remote Sensing Image Mis-registration**        │
│    - The paper deals with the effects of image mis-registration on pattern classification in remote sensing. It │
│ models data contamination and derives theoretical bounds on the classification accuracy loss resulting from     │
│ such contamination.                                                                                             │
│                                                                                                                 │
│ 6. **Low Congestion Online Routing and an Improved Mistake Bound for Online Prediction of Graph Labeling**      │
│    - This paper demonstrates a connection between graph routing and graph labeling prediction, presenting       │
│ improved mistake bounds for the latter based on routing efficiencies.                                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── 🔭 Searches Executed 1/1 ────────────────────────────────────────────╮
│                                                                                                                 │
│ QueryResultWithCollection(                                                                                      │
│     queries=['papers about classification'],                                                                    │
│     filters=[                                                                                                   │
│         [                                                                                                       │
│             BooleanPropertyFilter(                                                                              │
│                 property_name='about_classification',                                                           │
│                 operator=<ComparisonOperator.EQUALS: '='>,                                                      │
│                 value=True                                                                                      │
│             )                                                                                                   │
│         ]                                                                                                       │
│     ],                                                                                                          │
│     filter_operators='AND',                                                                                     │
│     collection='ArxivPapersDemo'                                                                                │
│ )                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 📊 No Aggregations Run                                                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── 📚 Sources ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  - object_id='1f899908-5460-4ad1-8d0c-39d406f30c8b' collection='ArxivPapersDemo'                                │
│  - object_id='ed693074-8db5-434c-a641-7c4f0855702f' collection='ArxivPapersDemo'                                │
│  - object_id='e6e85c70-041f-48c8-88f2-f3e2c8ac4ca3' collection='ArxivPapersDemo'                                │
│  - object_id='86ff0c94-c10f-4e11-9b14-a08b7a1b52c8' collection='ArxivPapersDemo'                                │
│  - object_id='7c78e036-4782-4756-b8f7-db40ca764a98' collection='ArxivPapersDemo'                                │
│  - object_id='728d9706-d87b-414b-90d2-075bba5bef89' collection='ArxivPapersDemo'                                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

   📊 Usage Statistics   
┌────────────────┬──────┐
│ LLM Requests:  │ 3    │
│ Input Tokens:  │ 6154 │
│ Output Tokens: │ 507  │
│ Total Tokens:  │ 6661 │
└────────────────┴──────┘

Total Time Taken: 9.30s

In [45]:
# Perform a query
response = qa.run(
    """
    How many papers are primarily about models?

    Hint: There is a property called 'paper_type' where the available values are: 'survey', 'model', 'system', 'analysis', 'resource', 'other'.
    """
)

# Print the response
response.display()

╭─────────────────────────────────────────────── 🔍 Original Query ───────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│     How many papers are primarily about models?                                                                 │
│                                                                                                                 │
│     Hint: There is a property called 'paper_type' where the available values are: 'survey', 'model', 'system',  │
│ 'analysis', 'resource', 'other'.                                                                                │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📝 Final Answer ────────────────────────────────────────────────╮
│                                                                                                                 │
│ There are 27 papers that are primarily about models.                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 🔭 No Searches Run                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 📊 Aggregations Run 1/1 ────────────────────────────────────────────╮
│                                                                                                                 │
│ AggregationResultWithCollection(                                                                                │
│     search_query=None,                                                                                          │
│     groupby_property=None,                                                                                      │
│     aggregations=[                                                                                              │
│         IntegerPropertyAggregation(property_name='paper_type', metrics=<NumericMetrics.COUNT: 'COUNT'>)         │
│     ],                                                                                                          │
│     filters=[                                                                                                   │
│         TextPropertyFilter(                                                                                     │
│             property_name='paper_type',                                                                         │
│             operator=<ComparisonOperator.EQUALS: '='>,                                                          │
│             value='model'                                                                                       │
│         )                                                                                                       │
│     ],                                                                                                          │
│     collection='ArxivPapersDemo'                                                                                │
│ )                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

   📊 Usage Statistics   
┌────────────────┬──────┐
│ LLM Requests:  │ 3    │
│ Input Tokens:  │ 3449 │
│ Output Tokens: │ 148  │
│ Total Tokens:  │ 3597 │
└────────────────┴──────┘

Total Time Taken: 3.04s

In [47]:
followup_response = qa.run(
    query="Can you select one or two of these papers and explain them in simple terms? I am not a data scientist.", context=response
)

followup_response.display()

╭─────────────────────────────────────────────── 🔍 Original Query ───────────────────────────────────────────────╮
│                                                                                                                 │
│ Can you select one or two of these papers and explain them in simple terms? I am not a data scientist.          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📝 Final Answer ────────────────────────────────────────────────╮
│                                                                                                                 │
│ Here are explanations for two papers primarily focused on models:                                               │
│                                                                                                                 │
│ 1. **Gaussian Process Topic Models (GPTMs)**:                                                                   │
│    - **Summary**: This paper introduces Gaussian Process Topic Models (GPTMs), which are advanced topic models. │
│ They leverage something called a 'kernel' among documents to extract interconnected topics. Think of them as a  │
│ sophisticated method to identify subjects within documents by considering how closely related these subjects    │
│ are across different documents.                                                                                 │
│    - **Key Concept**: GPTMs use a mathematical approach called Gaussian Process (GP) for embedding, where they  │
│ focus on the 'relations' or correlations among topics and documents. This involves a complex equation known as  │
│ the Sylvester equation, which helps in capturing these relationships.                                           │
│    - **Practical Use**: The paper demonstrates the effectiveness of GPTMs in accurately modeling topics and     │
│ their embeddings, showing improvements in how well they identify and represent topics in data.                  │
│                                                                                                                 │
│ 2. **Bayesian Exponential Family Projections for Coupled Data Sources**:                                        │
│    - **Summary**: The paper extends principal component analysis (PCA), a basic tool for data analysis, into    │
│ what's known as 'exponential family' models. This helps when data doesn't fit neatly into typical assumptions   │
│ like Gaussian distribution.                                                                                     │
│    - **Key Concept**: They introduce new methods for multi-view learning, which means learning from multiple    │
│ types of data sources simultaneously. The focus is on using a family of mathematical models that can better     │
│ capture the data's nature.                                                                                      │
│    - **Practical Use**: By providing a new family of models and inference strategies, the paper shows           │
│ improvement over older methods particularly in situations where the usual assumptions about data don't hold.    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── 🔭 Searches Executed 1/1 ────────────────────────────────────────────╮
│                                                                                                                 │
│ QueryResultWithCollection(                                                                                      │
│     queries=['papers primarily about models'],                                                                  │
│     filters=[                                                                                                   │
│         [                                                                                                       │
│             TextPropertyFilter(                                                                                 │
│                 property_name='paper_type',                                                                     │
│                 operator=<ComparisonOperator.EQUALS: '='>,                                                      │
│                 value='model'                                                                                   │
│             )                                                                                                   │
│         ]                                                                                                       │
│     ],                                                                                                          │
│     filter_operators='AND',                                                                                     │
│     collection='ArxivPapersDemo'                                                                                │
│ )                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 📊 No Aggregations Run                                                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── 📚 Sources ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  - object_id='f537c7fe-18e7-49c6-8621-521c1c144981' collection='ArxivPapersDemo'                                │
│  - object_id='6e9385a0-5189-4ce8-bd16-583d8195e5e3' collection='ArxivPapersDemo'                                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

   📊 Usage Statistics    
┌────────────────┬───────┐
│ LLM Requests:  │ 4     │
│ Input Tokens:  │ 10291 │
│ Output Tokens: │ 672   │
│ Total Tokens:  │ 10963 │
└────────────────┴───────┘

Total Time Taken: 17.21s

## Bonus: What NOT to do

Running multiple agents at the same time - this can cause conflicts (race conditions).

In [52]:
from weaviate.classes.config import Configure, Property, DataType

collection_name = "ArxivPapersDemo"

# Can delete the collection if you would like to (re)start fresh
client.collections.delete(collection_name)

client.collections.create(
    collection_name,
    description="A dataset that lists research paper titles and abstracts",
    properties=[
        Property(name="title", data_type=DataType.TEXT),
        Property(name="abstract", data_type=DataType.TEXT),
    ],
    vectorizer_config=[
        Configure.NamedVectors.text2vec_weaviate(
            name="default",
            source_properties=["title", "abstract"],
        )
    ]
)

papers_collection = client.collections.get(collection_name)
columns = papers_dataset[0]["properties"].keys()

with papers_collection.batch.fixed_size(100) as batch:
    for i, item in enumerate(papers_dataset):
        if i < 5:
            properties = {col: item["properties"][col] for col in columns}
            batch.add_object(properties=properties)


if papers_collection.batch.failed_objects:
    for fo in papers_collection.batch.failed_objects[:3]:
        print(fo.message)
        print(fo.object_)

len(papers_collection)

5

In [53]:
from weaviate.agents.transformation import TransformationAgent

responses = []
new_languages = ["spanish", "german", "italian"]

for lang in new_languages:

    prompt_task = f"""
    Create a {lang} version of the abstract
    """

    task = Operations.append_property(
        property_name=f"test_{lang}_abstract",
        data_type=DataType.TEXT,
        view_properties=["abstract"],
        instruction=prompt_task,
    )

    ta = TransformationAgent(
        client=client,
        collection=collection_name,
        operations=[task],
    )

    ta_response = ta.update_all()
    responses.append(ta_response)

print(responses)

[TransformationResponse(workflow_id='TransformationWorkflow-8f061ccf57fcfe44cdc4f7901134b0f0'), TransformationResponse(workflow_id='TransformationWorkflow-0b2420eebffa71c7f17bdd4e98a061ec'), TransformationResponse(workflow_id='TransformationWorkflow-4774969a9583d8145097339925db5605')]


In [54]:
for r in responses:
    get_ta_status(agent_instance=ta, workflow_id=r.workflow_id)

Waiting... Elapsed time: 4.67 seconds
Waiting... Elapsed time: 15.19 seconds
Waiting... Elapsed time: 25.64 seconds
Waiting... Elapsed time: 36.19 seconds
Waiting... Elapsed time: 46.68 seconds
Waiting... Elapsed time: 57.19 seconds
Waiting... Elapsed time: 67.68 seconds
Waiting... Elapsed time: 78.17 seconds
Waiting... Elapsed time: 88.87 seconds
Waiting... Elapsed time: 99.39 seconds
Waiting... Elapsed time: 109.90 seconds
Total time: 115.33 seconds
{'workflow_id': 'TransformationWorkflow-8f061ccf57fcfe44cdc4f7901134b0f0', 'status': {'batch_count': 1, 'end_time': '2025-03-25 12:26:59', 'start_time': '2025-03-25 12:25:04', 'state': 'completed', 'total_duration': 115.33162, 'total_items': 5}}
Total time: 9.85 seconds
{'workflow_id': 'TransformationWorkflow-0b2420eebffa71c7f17bdd4e98a061ec', 'status': {'batch_count': 1, 'end_time': '2025-03-25 12:25:15', 'start_time': '2025-03-25 12:25:05', 'state': 'completed', 'total_duration': 9.84898, 'total_items': 5}}
Total time: 12.28 seconds
{'w

In [55]:
response = papers_collection.query.fetch_objects(
    limit=50
)

properties = []
for o in response.objects:
    for p in o.properties.keys():
        if p not in properties:
            properties.append(p)
            print(f"Property {p} found in object UUID: {o.uuid}, adding to list")
    for p in properties:
        if p not in o.properties.keys():
            print(f"Property {p} not found in object UUID: {o.uuid}")
        if o.properties[p] is None or o.properties[p] == "":
            print(f"Property {p} is empty in object UUID: {o.uuid}")

Property abstract found in object UUID: 3eda93ee-dbd3-49ee-b7eb-fbd68f8f76e0, adding to list
Property title found in object UUID: 3eda93ee-dbd3-49ee-b7eb-fbd68f8f76e0, adding to list
Property test_german_abstract found in object UUID: 3eda93ee-dbd3-49ee-b7eb-fbd68f8f76e0, adding to list
Property test_spanish_abstract found in object UUID: 3eda93ee-dbd3-49ee-b7eb-fbd68f8f76e0, adding to list
Property test_italian_abstract found in object UUID: 3eda93ee-dbd3-49ee-b7eb-fbd68f8f76e0, adding to list
Property test_german_abstract is empty in object UUID: 3eda93ee-dbd3-49ee-b7eb-fbd68f8f76e0
Property test_spanish_abstract is empty in object UUID: ac502f5c-26e0-4fb7-a579-708313bf1171
Property test_german_abstract is empty in object UUID: e478240a-a5ac-4795-8e78-3d927dc06d32
Property test_german_abstract is empty in object UUID: ed90c10e-b748-48c3-80c0-634ce613f2ed
Property test_spanish_abstract is empty in object UUID: ed90c10e-b748-48c3-80c0-634ce613f2ed


## Further resources

- Blog: ["Introducing the Weaviate Transformation Agent"](https://weaviate.io/blog/transformation-agent)
- Documentation: [Weaviate Transformation Agent](https://weaviate.io/developers/agents/transformation)